In [ ]:
import cv2
import numpy as np

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_oval_shapes(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    horizontal_oval_contours = []

    # Iterate through contours and approximate shapes
    for contour in contours:
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Check if the contour is approximately oval
        if len(approx) >= 7:
            (x, y), (ma, Ma), angle = cv2.fitEllipse(contour)
            if Ma > ma and 10 < angle < 170:  # Ensure the major axis is horizontal and restrict angle range
                horizontal_oval_contours.append(contour)

    # Sort contours by the y-coordinate of their bounding rectangles
    horizontal_oval_contours.sort(key=lambda x: cv2.boundingRect(x)[1])

    # Keep only the two largest contours (ellipses) on the same horizontal line
    final_oval_contours = []
    for i in range(len(horizontal_oval_contours) - 1):
        y1 = cv2.boundingRect(horizontal_oval_contours[i])[1]
        y2 = cv2.boundingRect(horizontal_oval_contours[i + 1])[1]

        # Check if the ellipses are on the same horizontal line
        if abs(y1 - y2) < 20:  # Adjust the threshold as needed
            final_oval_contours.extend([horizontal_oval_contours[i], horizontal_oval_contours[i + 1]])

    # Draw ellipses around detected oval shapes
    for i in range(len(final_oval_contours)):
        ellipse = cv2.fitEllipse(final_oval_contours[i])

        # Scale the ellipse based on the average size of the eyes
        scaled_ellipse = (
            (int(ellipse[0][0]), int(ellipse[0][1])),
            (int(ellipse[1][0] * 0.7), int(ellipse[1][1] * 0.7)),  # Adjust the scale factor as needed
            ellipse[2]
        )

        cv2.ellipse(frame, scaled_ellipse, (0, 255, 0), 2)

    return frame

def main():
    # Open a connection to the camera (0 represents the default camera)
    cap = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around the detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Extract the face region
            face_region = frame[y:y + h, x:x + w]

            # Detect oval shapes within the upper half of the face region
            frame_with_ovals = detect_oval_shapes(face_region)

            # Replace the face region with the frame containing oval detections
            frame[y:y + h, x:x + w] = frame_with_ovals

        # Display the resulting frame
        cv2.imshow('Combined Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close the window
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
